In [2]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

In [21]:
from modules.data.conll2003.prc import conll2003_preprocess

In [22]:
data_dir = "/home/eartemov/ae/work/conll2003/"

In [13]:
conll2003_preprocess(data_dir)

## IO markup

### Train

In [3]:
from modules.data import bert_data

In [4]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTBiLSTMCRF

In [6]:
model = BERTBiLSTMCRF.create(
    len(data.train_ds.idx2label),
    lstm_dropout=0., crf_dropout=0.3)

In [7]:
from modules.train.train import NerLearner

In [8]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

2235023

In [ ]:
learner.fit(epochs=num_epochs)

### Predict

In [12]:
from modules.data.bert_data import get_data_loader_for_predict

In [13]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [14]:
preds = learner.predict(dl)

In [15]:
from sklearn_crfsuite.metrics import flat_classification_report

In [16]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [17]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [18]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[4:], digits=4)

In [20]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.9514    0.9509    0.9511      2016
         I_O     0.9968    0.9970    0.9969     41702
      I_MISC     0.9353    0.8974    0.9160      1257
       I_PER     0.9849    0.9825    0.9837      2856
       I_LOC     0.9697    0.9637    0.9667      1926

   micro avg     0.9917    0.9905    0.9911     49757
   macro avg     0.9676    0.9583    0.9629     49757
weighted avg     0.9916    0.9905    0.9910     49757



### Test

In [12]:
from modules.data.bert_data import get_data_loader_for_predict

In [24]:
dl = get_data_loader_for_predict(data, df_path="/home/eartemov/ae/work/conll2003/eng.testb.dev.csv")

In [25]:
preds = learner.predict(dl)

In [26]:
from sklearn_crfsuite.metrics import flat_classification_report

In [27]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [28]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [29]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[4:], digits=4)

In [30]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8988    0.9147    0.9067      2368
         I_O     0.9952    0.9917    0.9934     37573
      I_MISC     0.8163    0.8055    0.8108       910
       I_PER     0.9759    0.9770    0.9765      2698
       I_LOC     0.9170    0.9296    0.9233      1819

   micro avg     0.9822    0.9806    0.9814     45368
   macro avg     0.9206    0.9237    0.9221     45368
weighted avg     0.9823    0.9806    0.9814     45368

